In [ ]:
#Codingan ini berfungsi untuk membaca CSV, apakah csv bisa di baca atau tidak sebelum melakukan proses, hanya memastikan si CSV. 
#Ini digunakan ketika selalu error contohnya CSV failed. Kalau ada output kayak gini baru di testing ini code

def read_annotation_csv(csv_path):
    try:
        df = pd.read_csv(csv_path, sep=',', comment='#')
        expected_cols = ['channel', 'start_time', 'stop_time', 'label', 'confidence']
        if list(df.columns[:5]) != expected_cols:
            df.columns = expected_cols  # kalau header tidak dikenali otomatis
        print("File CSV berhasil dibaca dengan benar.")
        return df
    except Exception as e:
        raise ValueError(f"Gagal membaca file CSV: {e}")



#Disini pemanggilanya fungsi dan filenya
csv_file = '/content/aaaaaaac_s002_t000.csv'
df = read_annotation_csv(csv_file)


print("\n Isi DataFrame:")
print(df.head())


✅ File CSV berhasil dibaca dengan benar.

📋 Isi DataFrame:
  channel  start_time  stop_time label  confidence
0  FP1-F7      0.0000    16.0173  bckg         1.0
1  FP1-F7     16.0173   157.7731  cpsz         1.0
2  FP1-F7    157.7731   262.0000  bckg         1.0
3   F7-T3      0.0000    16.0173  bckg         1.0
4   F7-T3     16.0173   157.7731  cpsz         1.0


In [ ]:
import os
import numpy as np
import pandas as pd
import mne
import shutil

# === PARAMETER ===
fs = 256
seg_dur = 5
default_sph = 120  # baris ini: SPH default = 120 detik (2 menit)
default_sop = 60  # baris ini: SPH default = 60 detik (1 menit)

# === ROOT FOLDER ===
input_root = r'D:\eeg\v2.0.3\edf'
output_root = r'E:\eeg'

def process_folder(root, edf_files, csv_files, out_dir):
    try:
        edf_files = sorted([f for f in edf_files if f.endswith('.edf')])
        csv_files = sorted([f for f in csv_files if f.endswith('.csv')])

        raws = []
        annotations = []

        for edf_file, csv_file in zip(edf_files, csv_files):
            edf_path = os.path.join(root, edf_file)
            csv_path = os.path.join(root, csv_file)

            raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
            raws.append(raw)

            df = pd.read_csv(csv_path, sep=',', comment='#')
            df.columns = ['channel', 'start_time', 'stop_time', 'label', 'confidence']
            annotations.append(df)

        if not raws or not annotations:
            print(f"[SKIP] No valid data in folder: {root}")
            return

        combined_raw = mne.concatenate_raws(raws)
        data, times = combined_raw.get_data(return_times=True)
        combined_df = pd.concat(annotations, ignore_index=True)
        os.makedirs(out_dir, exist_ok=True)
        combined_df.to_csv(os.path.join(out_dir, "CSV_Gabungan_Token.csv"), index=False)

        df_seizure = combined_df[combined_df['label'] != 'bckg']
        if df_seizure.empty:
            print(f"[SKIP] No seizure in: {root}")
            return

        seizure_start = df_seizure['start_time'].min()
        total_window = default_sph + default_sop
        if seizure_start < total_window:
            total_window = seizure_start
            sph = int(total_window * 2 / 3)  #Jika seizure  terjadi < 180 detik (120+60), maka SPH & SOP akan disesuaikan. kalau seizure nya > 180 si SOP udah paten di 60 detik kalau SPHnya bisa adjust, maksudnya bisa dapat 24 segment bahkan lebih, tergantung nanti mau gunakan data yang seperti apa
            sop = int(total_window - sph) #Rasio penyesuaiannya: SPH = 2/3, SOP = 1/3 
            # print(f"Adjusted SPH/SOP → SPH: {sph}s, SOP: {sop}s")
        else:
            sph = default_sph
            sop = default_sop

            #paling" minimal itu 3 segment untuk memperkaya data terlebih dahulu. Ideal data yang bisa digunakan itu min 24 segment artiny 24x5=120 detik atau 2menit, 2 menit ini menandakan durasi SPH.

        total_samples = int(total_window * fs)
        start_idx = int((seizure_start - total_window) * fs)
        end_idx = int(seizure_start * fs)
        data_window = data[:, start_idx:end_idx]

        samples_per_seg = seg_dur * fs
        num_segments = total_samples // samples_per_seg
        if num_segments < 3:  #Line ini yang ngasih syarat paling minimal itu 3 segment kalau ga sampe 3 segment ya di skip
            print(f"[SKIP] Too few segments in: {edf_file}")
            return

        X, y = [], []
        for i in range(num_segments):
            s = i * samples_per_seg
            e = s + samples_per_seg
            X.append(data_window[:, s:e])
            y.append(1 if i >= sph // seg_dur else 0)

        X = np.stack(X)
        y = np.array(y)

        np.save(os.path.join(out_dir, "X.npy"), X)
        np.save(os.path.join(out_dir, "y.npy"), y)

        print(f"[OK] Saved combined X.npy | Segments: {X.shape[0]}")

    except Exception as e:
        print(f"[ERROR] Failed to process folder {root} → {e}")

def walk_and_process_all():
    for root, _, files in os.walk(input_root):
        edf_files = [f for f in files if f.endswith('.edf')]
        csv_files = [f for f in files if f.endswith('.csv')]
        if edf_files and csv_files:
            rel_path = os.path.relpath(root, input_root)
            out_dir = os.path.join(output_root, rel_path)
            process_folder(root, edf_files, csv_files, out_dir)

# === JALANKAN ===
walk_and_process_all()


[SKIP] Too few segments in: aaaaaajy_s001_t000.edf
[SKIP] Too few segments in: aaaaaajy_s002_t002.edf
[SKIP] Too few segments in: aaaaaajy_s003_t005.edf
[SKIP] Too few segments in: aaaaaajy_s004_t000.edf
[SKIP] Too few segments in: aaaaaayf_s001_t001.edf
[SKIP] Too few segments in: aaaaaayf_s002_t001.edf
[SKIP] No seizure in: D:\eeg\v2.0.3\edf\dev\aaaaaayf\s003_2003\01_tcp_ar
[SKIP] No seizure in: D:\eeg\v2.0.3\edf\dev\aaaaaayf\s004_2003\01_tcp_ar
[OK] Saved combined X.npy | Segments: 23
[SKIP] No seizure in: D:\eeg\v2.0.3\edf\dev\aaaaaazz\s002_2006\02_tcp_le
[SKIP] No seizure in: D:\eeg\v2.0.3\edf\dev\aaaaabep\s001_2003\02_tcp_le
[SKIP] No seizure in: D:\eeg\v2.0.3\edf\dev\aaaaabxe\s001_2003\02_tcp_le
[OK] Saved combined X.npy | Segments: 36
[SKIP] No seizure in: D:\eeg\v2.0.3\edf\dev\aaaaaclc\s001_2004\02_tcp_le
[SKIP] No seizure in: D:\eeg\v2.0.3\edf\dev\aaaaacqc\s001_2004\02_tcp_le
[SKIP] No seizure in: D:\eeg\v2.0.3\edf\dev\aaaaacqc\s002_2004\02_tcp_le
[OK] Saved combined X.npy | 

In [7]:
# === ROOT ASAL DAN TUJUAN ===
source_root = r"E:\eeg"
dest_root = r"E:\eeg\Full_Seizure"

# === FUNGSI CEK DAN SALIN ===
def copy_folders_with_npys():
    for root, dirs, files in os.walk(source_root):
        if "X.npy" in files and "y.npy" in files:
            # Tentukan jenis: dev/train/eval berdasarkan path
            if "dev" in root:
                subdir_type = "dev"
            elif "train" in root:
                subdir_type = "train"
            elif "eval" in root:
                subdir_type = "eval"
            else:
                continue  # skip jika tidak termasuk dev/train/eval

            # Buat path tujuan
            rel_path = os.path.relpath(root, os.path.join(source_root, subdir_type))
            dest_path = os.path.join(dest_root, subdir_type, rel_path)

            # Salin seluruh folder
            if not os.path.exists(dest_path):
                shutil.copytree(root, dest_path)
                print(f"[COPIED] {root} → {dest_path}")
            else:
                print(f"[SKIPPED] Sudah ada: {dest_path}")

# === JALANKAN ===
copy_folders_with_npys()


[COPIED] E:\eeg\dev\aaaaaazz\s001_2003\02_tcp_le → E:\eeg\Full_Seizure\dev\aaaaaazz\s001_2003\02_tcp_le
[COPIED] E:\eeg\dev\aaaaabxe\s002_2003\01_tcp_ar → E:\eeg\Full_Seizure\dev\aaaaabxe\s002_2003\01_tcp_ar
[COPIED] E:\eeg\dev\aaaaacqc\s003_2006\02_tcp_le → E:\eeg\Full_Seizure\dev\aaaaacqc\s003_2006\02_tcp_le
[COPIED] E:\eeg\dev\aaaaacyi\s001_2004\02_tcp_le → E:\eeg\Full_Seizure\dev\aaaaacyi\s001_2004\02_tcp_le
[COPIED] E:\eeg\dev\aaaaadkj\s002_2007\02_tcp_le → E:\eeg\Full_Seizure\dev\aaaaadkj\s002_2007\02_tcp_le
[COPIED] E:\eeg\dev\aaaaadkj\s003_2007\03_tcp_ar_a → E:\eeg\Full_Seizure\dev\aaaaadkj\s003_2007\03_tcp_ar_a
[COPIED] E:\eeg\dev\aaaaaewf\s001_2006\02_tcp_le → E:\eeg\Full_Seizure\dev\aaaaaewf\s001_2006\02_tcp_le
[COPIED] E:\eeg\dev\aaaaaewf\s003_2015\01_tcp_ar → E:\eeg\Full_Seizure\dev\aaaaaewf\s003_2015\01_tcp_ar
[COPIED] E:\eeg\dev\aaaaaexe\s001_2006\02_tcp_le → E:\eeg\Full_Seizure\dev\aaaaaexe\s001_2006\02_tcp_le
[COPIED] E:\eeg\dev\aaaaagbf\s001_2007\02_tcp_le → E:\eeg\Fu

In [8]:
# === ROOT ASAL DAN TUJUAN ===
source_root = r"E:\eeg"
dest_root = r"E:\eeg\Full_Seizure"

# === FUNGSI CEK DAN SALIN ===
def copy_folders_with_npys():
    for root, dirs, files in os.walk(source_root):
        if "X.npy" in files and "y.npy" in files:
            # Tentukan jenis: dev/train/eval berdasarkan path
            if "train" in root:
                subdir_type = "train"
            else:
                continue  # skip jika tidak termasuk dev/train/eval

            # Buat path tujuan
            rel_path = os.path.relpath(root, os.path.join(source_root, subdir_type))
            dest_path = os.path.join(dest_root, subdir_type, rel_path)

            # Salin seluruh folder
            if not os.path.exists(dest_path):
                shutil.copytree(root, dest_path)
                print(f"[COPIED] {root} → {dest_path}")
            else:
                print(f"[SKIPPED] Sudah ada: {dest_path}")

# === JALANKAN ===
copy_folders_with_npys()


[COPIED] E:\eeg\Full_Seizure\train\aaaaaaac\s002_2002\02_tcp_le → E:\eeg\Full_Seizure\train\..\Full_Seizure\train\aaaaaaac\s002_2002\02_tcp_le
[COPIED] E:\eeg\Full_Seizure\train\aaaaaaag\s005_2007\03_tcp_ar_a → E:\eeg\Full_Seizure\train\..\Full_Seizure\train\aaaaaaag\s005_2007\03_tcp_ar_a
[COPIED] E:\eeg\Full_Seizure\train\aaaaaaav\s001_2006\02_tcp_le → E:\eeg\Full_Seizure\train\..\Full_Seizure\train\aaaaaaav\s001_2006\02_tcp_le
[COPIED] E:\eeg\Full_Seizure\train\aaaaaacz\s006_2015\03_tcp_ar_a → E:\eeg\Full_Seizure\train\..\Full_Seizure\train\aaaaaacz\s006_2015\03_tcp_ar_a
[COPIED] E:\eeg\Full_Seizure\train\aaaaaalq\s001_2003\02_tcp_le → E:\eeg\Full_Seizure\train\..\Full_Seizure\train\aaaaaalq\s001_2003\02_tcp_le
[COPIED] E:\eeg\Full_Seizure\train\aaaaaalq\s002_2003\02_tcp_le → E:\eeg\Full_Seizure\train\..\Full_Seizure\train\aaaaaalq\s002_2003\02_tcp_le
[COPIED] E:\eeg\Full_Seizure\train\aaaaaasf\s001_2003\02_tcp_le → E:\eeg\Full_Seizure\train\..\Full_Seizure\train\aaaaaasf\s001_2003\0

In [ ]:
import os
import numpy as np
import pandas as pd

# === PARAMETER ===
source_root = r"E:\eeg\Full_Seizure"
output_csv = r"E:\eeg\segment_summary.csv"

# List untuk menampung data
summary_data = []

def copy_folders_with_npys():
    for root, dirs, files in os.walk(source_root):
        if "X.npy" in files and "y.npy" in files:
            x_path = os.path.join(root, "X.npy")
            y_path = os.path.join(root, "y.npy")

            try:
                X = np.load(x_path)
                y = np.load(y_path)

                segment_count = X.shape[0]
                channel_count = X.shape[1]
                sample_per_seg = X.shape[2]

                label_counts = dict(zip(*np.unique(y, return_counts=True)))
                label_0 = label_counts.get(0, 0)
                label_1 = label_counts.get(1, 0)

                summary_data.append({
                    "Path": root,
                    "Segmen": segment_count,
                    "Channel": channel_count,
                    "Sample_per_Segmen": sample_per_seg,
                    "Label_0": label_0,
                    "Label_1": label_1,
                    "Keterangan": f"File ini memiliki {segment_count} segmen"
                })

                print(f"[OK] {root} → {segment_count} segmen")

            except Exception as e:
                print(f"[ERROR] {x_path} → {e}")

    # Simpan ke CSV
    df_summary = pd.DataFrame(summary_data)
    df_summary.to_csv(output_csv, index=False)
    print(f"\n✅ Disimpan ke: {output_csv}")

# === JALANKAN ===
copy_folders_with_npys()


[OK] E:\eeg\Full_Seizure\dev\aaaaaazz\s001_2003\02_tcp_le → 23 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaabxe\s002_2003\01_tcp_ar → 36 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaacqc\s003_2006\02_tcp_le → 5 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaacyi\s001_2004\02_tcp_le → 20 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaadkj\s002_2007\02_tcp_le → 21 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaadkj\s003_2007\03_tcp_ar_a → 4 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaaewf\s001_2006\02_tcp_le → 36 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaaewf\s003_2015\01_tcp_ar → 36 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaaexe\s001_2006\02_tcp_le → 36 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaagbf\s001_2007\02_tcp_le → 36 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaagbf\s006_2016\01_tcp_ar → 24 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaagbf\s007_2016\01_tcp_ar → 36 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaagbf\s008_2016\01_tcp_ar → 36 segmen
[OK] E:\eeg\Full_Seizure\dev\aaaaagdr\s001_2007\02_tcp_le → 11 segmen
[OK] E:\eeg\Full_Sei

In [ ]:
import numpy as np

y = np.load('/content/y.npy')
print("Isi y.npy (label untuk tiap segmen):")
print(y.tolist())  # ubah ke list agar lebih mudah dibaca


In [ ]:
dest_root = r"E:\eeg\Full_Seizure"

x_path = r"E:\eeg\Full_Seizure\Full_Seizure\dev\aaaaaazz\s001_2003\02_tcp_le\x.npy"
y_path = r"E:\eeg\Full_Seizure\Full_Seizure\dev\aaaaaazz\s001_2003\02_tcp_le\y.npy"

x=np.load(x_path)
y=np.load(y_path)

print("Total window:", X.shape[0] * 5, "detik")
print("Segmen:", X.shape[0])
print("Channel:", X.shape[1])
print("Sample per segmen:", X.shape[2])

NameError: name 'X' is not defined